In [1]:
%matplotlib inline


신경망(Neural Networks)
=======================

신경망은 ``torch.nn`` 패키지를 사용하여 생성할 수 있습니다.

지금까지 ``autograd`` 를 살펴봤는데요, ``nn`` 은 모델을 정의하고 미분하는데
``autograd`` 를 사용합니다.
``nn.Module`` 은 계층(layer)과 ``output`` 을 반환하는 ``forward(input)``
메서드를 포함하고 있습니다.

숫자 이미지를 분류하는 신경망을 예제로 살펴보겠습니다:

![convnet](images/mnist.png)

이는 간단한 피드-포워드 네트워크(Feed-forward network)입니다. 입력(input)을 받아
여러 계층에 차례로 전달한 후, 최종 출력(output)을 제공합니다.

신경망의 전형적인 학습 과정은 다음과 같습니다:

- 학습 가능한 매개변수(또는 가중치(weight))를 갖는 신경망을 정의합니다.
- 데이터셋(dataset) 입력을 반복합니다.
- 입력을 신경망에서 처리합니다.
- 손실(loss; 출력이 정답으로부터 얼마나 떨어져있는지)을 계산합니다.
- 변화도(gradient)를 신경망의 매개변수들에 역으로 전파합니다.
- 신경망의 가중치를 갱신합니다. 일반적으로 다음과 같은 간단한 규칙을 사용합니다:
  ``가중치(wiehgt) = 가중치(weight) - 학습율(learning rate) * 변화도(gradient)``

신경망 정의하기
---------------

이제 신경망을 정의해보겠습니다:



In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module): 
    #모델 정의. nn.Module을 상속한다. forward()를 구현해야 한다.

    def __init__(self): #생성자
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        #nn.Sequential과 OrderedDict를 사용해 컨테이너로 감쌀 수도 있다.

    def forward(self, x): #forward를 정의하면, autograd로 backward는 자동으로 정의된다. #input tensor를 그대로 사용할 수 있다.
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2)) #Activate function으로 Relu 적용 후 Max pooling
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x)) #shape를 변경한다. -1을 입력하면, 자동으로 형태를 맞춰준다.
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension #배치를 제외한 input의 차원
        num_features = 1
        for s in size:
            num_features *= s #각 차원을 곱해, 수를 구한다.
        return num_features #x의 모든 차원의 수와 같다.


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


``forward`` 함수만 정의하고 나면, (변화도를 계산하는) ``backward`` 함수는
``autograd`` 를 사용하여 자동으로 정의됩니다.
``forward`` 함수에서는 어떠한 Tensor 연산을 사용해도 됩니다.

모델의 학습 가능한 매개변수들은 ``net.parameters()`` 에 의해 반환됩니다.



In [3]:
params = list(net.parameters()) #모델의 모든 변수들이 들어 있다.
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


임의의 32x32 입력값을 넣어보겠습니다.
Note: 이 신경망(LeNet)의 입력은 32x32입니다. 이 신경망에 MNIST 데이터셋을
사용하기 위해서는, 데이터셋의 이미지를 32x32로 크기를 변경해야 합니다.



In [4]:
input = torch.randn(1, 1, 32, 32) #32 x 32 랜덤
out = net(input) #output은 마지막 fc의 output_dim 인 10이 된다.
print(out)

tensor([[-0.1332, -0.0018, -0.0187,  0.1631, -0.0282, -0.0398,  0.0413, -0.1152,
         -0.1237, -0.1152]], grad_fn=<ThAddmmBackward>)


모든 매개변수의 변화도 버퍼(gradient buffer)를 0으로 설정하고, 무작위 값으로
역전파를 합니다:



In [5]:
net.zero_grad() #버퍼를 0으로. 보통의 training에서는 weight 값들이 변화하므로, 버퍼를 지운 이후에 진행해야 한다.
out.backward(torch.randn(1, 10)) #랜덤값으로 역전파. 보통은 파라미터 없이 그냥 backward() 하면 된다.

<div class="alert alert-info"><h4>Note</h4><p>``torch.nn`` 은 미니 배치(mini-batch)만 지원합니다. ``torch.nn`` 패키지
    전체는 하나의 샘플이 아닌, 샘플들의 미니배치만을 입력으로 받습니다.

    예를 들어, ``nnConv2D`` 는 ``nSamples x nChannels x Height x Width`` 의
    4차원 Tensor를 입력으로 합니다.

    만약 하나의 샘플만 있다면, ``input.unsqueeze(0)`` 을 사용해서 가짜 차원을
    추가합니다.</p></div>

계속 진행하기 전에, 지금까지 살펴봤던 것들을 다시 한번 요약해보겠습니다.

**Recap:**
  -  ``torch.Tensor`` - ``backward()`` 같은 autograd 연산을 지원하는
     *다차원 배열* 입니다. 또한 tensor에 대한 *변화도(Gradient)를 갖고* 있습니다.
  -  ``nn.Module`` - 신경망 모듈. *매개변수를 캡슐화(Encapsulation)하는 간편한
     방법* 으로, GPU로 이동, 내보내기(exporting), 불러오기(loading) 등의 작업을
     위한 헬퍼(helper)를 제공합니다.
  -  ``nn.Parameter`` - Tensor의 한 종류로, ``Module`` *에 속성으로 할당될 때
     자동으로 매개변수로 등록* 됩니다.
  -  ``autograd.Function`` - *autograd 연산의 전방향과 역방향 정의* 를 구현합니다.
     모든 ``Tensor`` 연산은 하나 이상의 ``Function`` 노드를 생성하며, 각 노드는
     ``Tensor`` 를 생성하고 *이력(History)을 부호화* 하는 함수들과 연결하고 있습니다.


**지금까지 우리가 다룬 내용은 다음과 같습니다:**
  -  신경망을 정의하는 것
  -  입력을 처리하고 ``backward`` 를 호출하는 것

**더 살펴볼 내용들은 다음과 같습니다:**
  -  손실을 계산하는 것
  -  신경망의 가중치를 갱신하는 것

손실 함수 (Loss Function)
-------------------------
손실 함수는 (output, target)을 한 쌍(pair)의 입력으로 받아, 출력(output)이
정답(target)으로부터 얼마나 떨어져있는지를 추정하는 값을 계산합니다.

nn 패키지에는 여러가지의 [손실 함수들](http://pytorch.org/docs/nn.html#loss-functions)
이 존재합니다.
간단한 손실 함수로는 출력과 대상간의 평균자승오차(mean-squared error)를 계산하는
``nn.MSEloss`` 가 있습니다.

예를 들면:



In [6]:
output = net(input)
target = torch.arange(1, 11, dtype=torch.float)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output

criterion = nn.MSELoss() #손실함수. 평균제곱 손실

loss = criterion(output, target) #target에 대한 output의 손실을 MSE로 구한다.
print(loss)

tensor(39.0727, grad_fn=<MseLossBackward>)


이제 ``.grad_fn`` 속성을 사용하여 ``loss`` 를 역방향에서 따라가다보면,
이러한 모습의 연산 그래프를 볼 수 있습니다.

::

    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
          -> view -> linear -> relu -> linear -> relu -> linear
          -> MSELoss
          -> loss

따라서, ``loss.backward()`` 를 실행할 때, 전체 그래프는 손실(loss)에 대해
미분되며, 그래프 내의 ``requires_grad=True`` 인 모든 Tensor는 변화도(Gradient)가
누적된 ``.grad`` Tensor를 갖게 됩니다.

설명을 위해, 역전파의 몇 단계를 따라가보겠습니다:



In [7]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

역전파(Backprop)
----------------
오차(error)를 역전파하기 위해 할 일은 ``loss.backward()`` 이 전부입니다.
기존 변화도를 지우는 작업이 필요한데, 그렇지 않으면 변화도가 기존의 것에
누적되기 때문입니다.


이제 ``loss.backward()`` 를 호출하여 역전파 전과 후에 conv1의 bias gradient를
살펴보겠습니다.



In [8]:
net.zero_grad()     # zeroes the gradient buffers of all parameters #기존의 값을 제거해 준다. 버퍼를 비워준다. 그렇지 않으면 누적된다.

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad) #편향

loss.backward() #전체 손실에 대해 미분된다. #오차를 역전파한다.

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0192,  0.0613,  0.0441, -0.0113, -0.0218,  0.0203])


지금까지 손실 함수를 어떻게 사용하는지를 살펴봤습니다.

**더 읽어보기:**

  신경망 패키지(neural network package)에는 심층 신경망(deep neural network)을
  구성하는 다양한 모듈과 손실함수가 포함되어 있습니다.
  전체 목록은 [이 문서](http://pytorch.org/docs/nn) 에 있습니다.

**이제 더 살펴볼 내용은 다음과 같습니다:**

  -  신경망의 가중치를 갱신하는 것

가중치 갱신
------------------
실제로 많이 사용되는 가장 단순한 갱신 규칙은 확률적 경사하강법(SGD; Stochastic
Gradient Descent)입니다:

     ``가중치(wiehgt) = 가중치(weight) - 학습율(learning rate) * 변화도(gradient)``

간단한 Python 코드로 이를 구현해볼 수 있습니다:

.. code:: python

    learning_rate = 0.01
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)

그러나, 신경망을 구성할 때 SGD, Nesterov-SGD, Adam, RMSProp 등과 같은 다양한
갱신 규칙을 사용하고 싶을 수 있습니다. 이를 위해서 ``torch.optim`` 라는 작은
패키지에 이러한 방법들을 모두 구현해두었습니다. 사용법은 매우 간단합니다:



In [9]:
import torch.optim as optim

# Optimizer를 생성합니다.
optimizer = optim.SGD(net.parameters(), lr=0.01) #평균경사하강법

# 학습 과정(training loop)에서는 다음과 같습니다:
optimizer.zero_grad()   # zero the gradient buffers #버퍼를 비워준다. 비워주지 않으면 값이 누적되어 다른 값이 나온다.
output = net(input) #model에 input을 입력해, output을 받는다.
loss = criterion(output, target) #손실함수로 손실을 구한다.
loss.backward() #역전파 한다.
optimizer.step()    # Does the update #최적화함수로 변수를 업데이트 한다.

.. Note::

      ``optimizer.zero_grad()`` 를 사용하여 수동으로 변화도 버퍼를 0으로 설정하는
      것에 유의하세요. 이는 `역전파(Backprop)`_ 섹션에서 설명한 것처럼 변화도가
      누적되기 때문입니다.

